In [116]:
import pandas as pd
import gensim
from nltk import word_tokenize
import numpy as np
import annoy

In [168]:
data = pd.read_csv('/Users/Valeriya/Downloads/root/LABS/thequestion-hack/data/thequestion.csv')

In [169]:
data

,id,title,topic_ids,answers
0,1,Насколько может быть сильным самовнушение?,"{24,26,11}",Сила внушения может быть очень большой. Всем х...
1,2,Часто ли болеют полярники \r\nна работе?,"{27,11}","Вообще не болеют. На полюсах так холодно, что ..."
2,3,Где-нибудь кроме России еще боятся сквозняка?,"{2,26,11,10,59}",Сквозняка просто нигде не бывает кроме как в Р...
3,4,"Правда ли, что библиотечная печать всегда став...","{18,7003,10}","Это старая традиция, связанная с технологией п..."
4,5,Достаточно ли в России бомбоубежищ?,"{6,29,25,164}","Нет, недостаточно. Хотя, откровенно говоря, я ..."
5,6,Почему облака не падают на землю?,"{66,51,74,34,59}",Облака — это водяной пар и мельчайшие частицы ...
6,7,Каковы плюсы и минусы централизации России вок...,"{35,29,164}",Концентрация населения и экономической деятель...
7,8,Может ли человек чихать во сне?,"{33,11}",Чихание возникает вследствие раздражения слизи...
8,9,"Платят ли космонавтам командировочные, и кем о...","{20,34}","Да, командировочные нам платят, между прочим в..."
9,10,Можно ли в возрасте 40 лет научиться играть на...,"{13,18049}",В таком возрасте можно научиться играть только...


In [177]:
ques = list(data['title'])

In [178]:
ques

['Насколько может быть сильным самовнушение?',
 'Часто ли болеют полярники \r\nна работе?',
 'Где-нибудь кроме России еще боятся сквозняка?',
 'Правда ли, что библиотечная печать всегда ставится на 17 странице книги?',
 'Достаточно ли в России бомбоубежищ?',
 'Почему облака не падают на землю?',
 'Каковы плюсы и минусы централизации России вокруг Москвы?',
 'Может ли человек чихать во сне?',
 'Платят ли космонавтам командировочные, и кем они работают на Земле?',
 'Можно ли в возрасте 40 лет научиться играть на пианино и достичь высокого уровня?',
 'Почему антисептик разрушает клетки бактерий, но не разрушает клетки человека?',
 'Правда ли, что в Чечне огромные пособия на детей?',
 'Останется что-то будущим поколениям из Фонда будущих поколений?',
 'Какая самая популярная статья в «Википедии»?',
 'Почему человек иногда просыпается раньше будильника?',
 'Чем с точки зрения современной науки является депрессия и какие есть современные способы ее лечения?',
 'Как может быть отрицательная п

In [191]:
tokenized_ques = [word_tokenize(sent) for sent in ques]

In [192]:
model = gensim.models.Word2Vec(tokenized_ques, size=600, window=5, min_count=2, workers=4)

In [193]:
model.build_vocab

<bound method Word2Vec.build_vocab of <gensim.models.word2vec.Word2Vec object at 0x27de780f0>>

In [195]:
# model.wv['ли']

In [196]:
X_word2vec=[]
for i in ques:
    ques_vec= []
    tokens = word_tokenize(i)
    for w in tokens:
        if w in model:
            ques_vec.append(model[w])
    try:
        mean_vector = sum(ques_vec) / len(ques_vec)
        X_word2vec.append(mean_vector)   
    except:
        pass

In [197]:
def get_question_vec(ques):
    tokens = word_tokenize(ques)
    ques_vec= []
    for w in tokens:
        if w in model:
            ques_vec.append(model[w])
    try:
        mean_vector = sum(ques_vec) / len(ques_vec)
        return mean_vector
    except:
        return []

In [198]:
my_ques = 'Где-нибудь кроме России еще боятся сквозняка?'

In [199]:
v = get_question_vec(my_ques)
# from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# for i, ind in enumerate(X_word2vec):
#     print(i)
#     print(ind)
#     break

In [110]:
# max_similar = 0
# ind_max = 0
# for ind, i in enumerate(X_word2vec[:100]):
#     i = i.reshape(1,-1)
#     sim = cosine_similarity(i, v)
#     if sim > max_similar:
#         max_similar = sim
#         ind_max = ind

In [215]:
# matrix = np.matrix(X_word2vec)

In [201]:
from annoy import AnnoyIndex
f = 600
t = AnnoyIndex(f)  # Length of item vector that will be indexed
for i in range(len(X_word2vec)):
    v = X_word2vec[i]
    t.add_item(i, v)

In [202]:
t.build(20)

True

In [206]:
t.get_nns_by_vector(v, 10)

[186709, 180268, 181199, 167784, 129473, 100650, 50144, 109671, 40565, 6993]

In [214]:
model.wv.most_similar('вопрос')

[('ответ', 0.8622888326644897),
 ('ответить', 0.7912372350692749),
 ('мой', 0.7222918272018433),
 ('вопросы', 0.71379554271698),
 ('фильм', 0.7114942073822021),
 ('твой', 0.7100188732147217),
 ('отвечать', 0.7064404487609863),
 ('спросить', 0.696880578994751),
 ('TheQuestion', 0.6967288851737976),
 ('задать', 0.6935933232307434)]

In [208]:
for i in t.get_nns_by_vector(v, 10):
    print(ques[i])

К какому эксперту Q обратиться за помощью в разрешении проблемы получения паспорта и гражданства РФ?
Почему так активно критикуются ютуб каналы с прохождением игр и твич?
Если я получила среднее-специльное образование в России в области музыки, есть ли возможность учиться в Новой Зеландии по совершенно другому направлению?
Что вы считаете достойно того чтобы быть запечатлено на вашем теле в виде татуировки?
Как лучше всего ухаживать за зубами и полостью рта?
Как жить, если ты гендерно-неконформный мужчина: неинициативный, женственный, страдаешь перепадами настроения и ничего не можешь с собой сделать?
Какие видео вызвали у вас наибольшее эстетическое удовольствие?
Где искать новые знакомства?
Какие у вас любимые зарубежные сериалы?
Какие у Вас любимые дипломатические слова?


In [152]:
##### _____________________________ ####

In [154]:
# answ = norm.dot(v)

In [155]:
# list_a = answ.tolist()[0]

In [156]:
# max_num = max(list_a)

In [157]:
# ind = list_a.index(max_num)

In [158]:
# ques[ind]

In [159]:
# max_num

In [160]:
# from numpy.linalg import norm
# norm(matrix[7899])

In [161]:
# norm(v)

In [92]:
# ? norm